# `jku.wad` Tournament

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import random
import yaml
from collections import defaultdict

from doom_arena import VizdoomMPEnv

## Set up tree

In [3]:
class Tournament:
    def __init__(self, submission_dir: str, max_players_per_match: int = 4):
        self.submissions = {}
        # TODO what is the default naming pattern?
        for filename in os.listdir(submission_dir):
            if filename.endswith(".onnx"):
                self.submissions[filename] = os.path.join(submission_dir, filename)

        self.round = 0
        self.init_max_players_per_match = max_players_per_match
        self.max_players_per_match = max_players_per_match
        self.adjust_max_players_per_match(len(self.submissions))

    def adjust_max_players_per_match(self, players_left: int):
        if players_left % self.max_players_per_match == 0:
            return

        for i in range(self.max_players_per_match, 1, -1):
            if players_left % i != 1:
                self.max_players_per_match = i
                break
        else:
            self.max_players_per_match = 2

    def create_initial_round(self):
        self.round = 0
        self.max_players_per_match = self.init_max_players_per_match
        players = list(self.submissions.keys())
        random.shuffle(players)

        while len(players) % self.max_players_per_match != 0:
            players.append(None)

        matches = [
            players[i : i + self.max_players_per_match]
            for i in range(0, len(players), self.max_players_per_match)
        ]
        return matches

    def advance_round(self, winners):
        self.round += 1
        if len(winners) == 1:
            return winners
        next_round_players = winners.copy()
        self.adjust_max_players_per_match(len(next_round_players))

        while len(next_round_players) % self.max_players_per_match != 0:
            next_round_players.append(None)

        next_matches = [
            next_round_players[i : i + self.max_players_per_match]
            for i in range(0, len(next_round_players), self.max_players_per_match)
        ]
        return next_matches

In [4]:
def mock_listdir(_):
    return [f"student_{i:02}.onnx" for i in range(1, 21)]


os.listdir = mock_listdir

## Run tournament

In [5]:
tourn = Tournament("mock_dir")

In [6]:
def load_onnx(path):
    # TODO
    def rnd_policy(obs):
        act = 2
        return act

    return rnd_policy


def play_match(
    players, submissions, num_episodes: int = 1, record_replay: bool = False
):
    players = [p for p in players if p is not None]
    # TODO student env configs
    # player_configs = [yaml.safe_load(submissions[p]) for p in players]
    player_configs = [{} for p in players]
    player_agents = {p: load_onnx(submissions[p]) for p in players}
    env = VizdoomMPEnv(
        num_players=len(players),
        num_bots=0,
        doom_map="TRNM",
        extra_state=[pcfg.get("extra_state", None) for pcfg in player_configs],
        crosshair=[pcfg.get("crosshair", False) for pcfg in player_configs],
        hud=[pcfg.get("hud", "full") for pcfg in player_configs],
        episode_timeout=2000,
    )
    if record_replay:
        env.enable_replay()

    results = []
    for _ in range(num_episodes):
        ep_return = {k: 0.0 for k in players}
        obs = env.reset()
        done = False
        while not done:
            player_acts = [agnt(obs) for agnt in player_agents.values()]
            obs, rwd, done, _ = env.step(player_acts)
            ep_return = {k: ep_return[k] + rwd[i] for i, k in enumerate(ep_return)}

        results.append(ep_return)

    # average episodes per player
    match_returns = {p: 0.0 for p in players}
    for result in results:
        for p in players:
            match_returns[p] += result[p]
    match_returns = {p: total / num_episodes for p, total in match_returns.items()}
    leaderboard = sorted(match_returns.items(), key=lambda x: x[1], reverse=True)
    if record_replay:
        return leaderboard, env.get_player_replays()
    else:
        return leaderboard


def play_round(
    matches, submissions, num_episodes: int = 1, record_replay: bool = False
):
    round_winners, round_leaderboards = [], []
    round_replays = []
    for players in matches:
        results = play_match(players, submissions, num_episodes)
        if record_replay:
            leaderboard, replays = results
            round_replays.append(replays)
        else:
            leaderboard = results
        round_winners.append(leaderboard[0][0])
        round_leaderboards.append(leaderboard)

    return round_winners, round_leaderboards


def leaderboard_reduce(leaderboards):
    scores = defaultdict(lambda: [0.0, 0])

    for round_group in leaderboards:
        for match in round_group:
            for student, score in match:
                scores[student][0] += float(score)  # Accumulate score
                scores[student][1] += 1  # Increment count

    # Convert to dict of tuples for readability
    return {k: (v[0], v[1]) for k, v in scores.items()}

In [7]:
current_matches = tourn.create_initial_round()

return_leaderboard = []
round_matches = [current_matches]
round_winners = []
while len(current_matches) > 1:
    print(f"\nNow playing: round {tourn.round}:")
    for match in current_matches:
        print("\tMatch:", match)
    winners, leaderboards = play_round(current_matches, tourn.submissions)
    return_leaderboard.append(leaderboards)
    round_winners.append(winners)
    print("\tWinners:", winners)
    # advange tree to next round
    current_matches = tourn.advance_round(winners)
    round_matches.append(current_matches)

print(f"\nNow playing: final round:")
final_match = current_matches[0]
print("\tMatch:", match)
winners, leaderboards = play_round(current_matches, tourn.submissions)
return_leaderboard.append(leaderboards)
round_winners.append(winners)
print("\tWinner:", winners)


Now playing: round 0:
	Match: ['student_19.onnx', 'student_11.onnx', 'student_13.onnx', 'student_10.onnx']
	Match: ['student_07.onnx', 'student_03.onnx', 'student_12.onnx', 'student_08.onnx']
	Match: ['student_18.onnx', 'student_04.onnx', 'student_14.onnx', 'student_01.onnx']
	Match: ['student_20.onnx', 'student_15.onnx', 'student_02.onnx', 'student_09.onnx']
	Match: ['student_06.onnx', 'student_17.onnx', 'student_05.onnx', 'student_16.onnx']
Host 43229
Player 43229


Network game synchronization timeout: 60s.
Contacting host: /Network game synchronization timeout: 60s.
Contacting host: /Network game synchronization timeout: 60s.
Contacting host: /Network game synchronization timeout: 60s.
Got connect from node 1.                
Got connect from node 2.                
Got connect from node 3.                
Waiting for players: .... [ 4/ 4]
Sending all here.                       

Total players: 4                        

Waiting for other players: ..   [ 2/ 4]
Total players: 4                        

Waiting for other players: ...  [ 3/ 4]
Total players: 4                        

Waiting for other players: .... [ 4/ 4]
Console player number: 1                
Received All Here, sending ACK.         
Console player number: 2                
Received All Here, sending ACK.         
Console player number: 3                
Received All Here, sending ACK.         

Go                                      
Total players: 4                        






Host 34207
Player 34207


Network game synchronization timeout: 60s.
Waiting for players: .    [ 1/ 4]Network game synchronization timeout: 60s.
Contacting host: /Network game synchronization timeout: 60s.
Contacting host: /Network game synchronization timeout: 60s.
Got connect from node 1.                
Got connect from node 2.                
Got connect from node 3.                
Waiting for players: .... [ 4/ 4]
Sending all here.                       

Total players: 4                        

Waiting for other players: ..   [ 2/ 4]
Total players: 4                        

Waiting for other players: ...  [ 3/ 4]
Total players: 4                        

Waiting for other players: .... [ 4/ 4]
Console player number: 1                
Received All Here, sending ACK.         
Console player number: 2                
Received All Here, sending ACK.         
Console player number: 3                
Received All Here, sending ACK.         

Go                                      
Total players: 4          





Host 36703
Player 36703


...




Host 36505
Player 36505


Network game synchronization timeout: 60s.
Waiting for players: .  [ 1/ 2]Network game synchronization timeout: 60s.
Got connect from node 1.                
Waiting for players: .. [ 2/ 2]
Sending all here.                       

Total players: 2                        

Waiting for other players: .. [ 2/ 2]
Console player number: 1                
Received All Here, sending ACK.         

Go                                      
Total players: 2                        

Console player number: 1                
Received All Here, sending ACK.         
Received "Go."                          

Total players: 2                        

Found WhoAmI (node 1, player 1)         
Found P1 (node 1, player 2)             
Found P1 (node 0, player 2)             





	Winners: ['student_19.onnx', 'student_20.onnx']

Now playing: final round:
	Match: ['student_20.onnx', 'student_06.onnx', None]

Host 37175
Player 37175


Network game synchronization timeout: 60s.
Waiting for players: .  [ 1/ 2]Network game synchronization timeout: 60s.
Got connect from node 1.                
Waiting for players: .. [ 2/ 2]
Sending all here.                       

Total players: 2                        

Waiting for other players: .. [ 2/ 2]
Console player number: 1                
Received All Here, sending ACK.         

Go                                      
Total players: 2                        

Console player number: 1                
Received All Here, sending ACK.         
Received "Go."                          

Total players: 2                        

Found WhoAmI (node 1, player 1)         
Found P1 (node 1, player 2)             
Found P1 (node 0, player 2)             





	Winner: ['student_19.onnx']

